In [27]:
import torch
torch.cuda.is_available()

True

In [1]:
# core libraries
! pip install torch torchvision numpy matplotlib
# for attacks
! pip install cleverhans foolbox


  Using cached pycodestyle-2.12.1-py2.py3-none-any.whl.metadata (4.5 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------- -------------- 1.0/1.7 MB 12.5 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 5.3 MB/s eta 0:00:00
Using cached pycodestyle-2.12.1-py2.py3-none-any.whl (31 kB)
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---- ----------------------------------- 0.8/6.9 MB 3.1 MB/s eta 0:00:02
   ------- -------------------------------- 1.3/6.9 MB 2.9 MB/s eta 0:00:02
   ------------ --------------------------- 2.1/6.9 MB 3.2 MB/s eta 0:00:02
   ---------------- ----------------------- 2.9/6.9 MB 3.2 MB/s eta 0:00:02
   ------------------- -------------------- 3.4/6.9 MB 3.1 MB/s eta 0:00:02
   ------------------------ --------------- 4.2/6.9 MB 3.1 MB/s eta 0:00:01
   --------------------------- ------------ 4.7/6.9 MB 3.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.4.1 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.


In [3]:
! pip install transformers datasets


  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/10.0 MB 7.5 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/10.0 MB 4.7 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/10.0 MB 4.3 MB/s eta 0:00:02
   ------------- -------------------------- 3.4/10.0 MB 3.9 MB/s eta 0:00:02
   --------------- ------------------------ 3.9/10.0 MB 3.7 MB/s eta 0:00:02
   ------------------ --------------------- 4.7/10.0 MB 3.7 MB/s eta 0:00:02
   -------------------- ------------------- 5.2/10.0 MB 3.6 MB/s eta 0:00:02
   ------------------------ --------------- 6.0/10.0 MB 3.5 MB/s eta 0:00:02
   --------------------------- ------------ 6.8/10.0 MB 3.5 MB/s eta 0:00:01
   ----------------------------- ---------- 7.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.4.1 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.


In [2]:
from transformers import ViTImageProcessor, ViTForImageClassification

MODEL_ID = "farleyknight-org-username/vit-base-mnist"
processor = ViTImageProcessor.from_pretrained(MODEL_ID)  # handles resize & normalize :contentReference[oaicite:2]{index=2}
model     = ViTForImageClassification.from_pretrained(MODEL_ID)
model.eval()


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [3]:
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Grayscale, Resize, ToTensor, Normalize

transform = Compose([
    Resize(224),                        # ViT input size :contentReference[oaicite:4]{index=4}
    Grayscale(num_output_channels=3),   # convert to RGB
    ToTensor(),
    Normalize(mean=[0.5]*3, std=[0.5]*3)  # ViT normalization :contentReference[oaicite:5]{index=5}
])

train_ds = MNIST(root="data", train=True,  download=True, transform=transform)
test_ds  = MNIST(root="data", train=False, download=True, transform=transform)


In [4]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=2)
test_loader  = DataLoader(test_ds,  batch_size=64, shuffle=False, num_workers=2)


In [5]:
import torch
from tqdm import tqdm

correct, total = 0, 0
for batch in tqdm(test_loader, desc="Eval"):
    pixel_values = batch[0]  # shape: [B, 3, 224, 224]
    labels       = batch[1]
    with torch.no_grad():
        logits = model(pixel_values).logits  # ViTForImageClassification returns logits :contentReference[oaicite:7]{index=7}
    preds = logits.argmax(dim=-1)
    correct += (preds == labels).sum().item()
    total   += labels.size(0)
    break  # just one batch for smoke test

print(f"Batch accuracy: {correct/total:.2%}")


Eval:   0%|          | 0/157 [00:11<?, ?it/s]

Batch accuracy: 100.00%


In [7]:
# train.py
import torch
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Resize, Grayscale, ToTensor, Normalize

# 0. Device selection
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 1. Data transforms & datasets
transform = Compose([
    Resize(224),
    Grayscale(num_output_channels=3),
    ToTensor(),
    Normalize(mean=[0.5]*3, std=[0.5]*3)
])
train_ds = MNIST("data", train=True,  download=True, transform=transform)
test_ds  = MNIST("data", train=False, download=True, transform=transform)

# 2. DataLoaders with pinned memory for faster transfers
loader_kwargs = {
    "batch_size": 32,
    "shuffle": True,
    "num_workers": 4,     # parallel data loading
    "pin_memory": True    # page-locked memory for non-blocking copies
}
train_loader = DataLoader(train_ds, **loader_kwargs)
test_loader  = DataLoader(test_ds,  batch_size=64, shuffle=False,
                          num_workers=4, pin_memory=True)

# 3. Model initialization & move to GPU
model = ViTForImageClassification.from_pretrained(
    "farleyknight-org-username/vit-base-mnist"
)
model.to(device)           # move model parameters/buffers to GPU
model.train()

# 4. Optimizer & loss
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
criterion = torch.nn.CrossEntropyLoss()

# 5. Training loop with GPU data transfers
epochs = 5
for epoch in range(epochs):
    epoch_loss = 0.0
    for pixel_values, labels in train_loader:
        # move batch to GPU with non-blocking pin-memory transfer
        pixel_values = pixel_values.to(device, non_blocking=True)
        labels       = labels.to(device,       non_blocking=True)

        optimizer.zero_grad()
        logits = model(pixel_values=pixel_values).logits
        loss   = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} — Loss: {epoch_loss/len(train_loader):.4f}")


Using device: cuda
Epoch 1/5 — Loss: 0.0412
Epoch 2/5 — Loss: 0.0264
Epoch 3/5 — Loss: 0.0199
Epoch 4/5 — Loss: 0.0176
Epoch 5/5 — Loss: 0.0161


In [12]:
! pip install torchattacks

  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.13.2-py3-none-any.whl (45 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.1.0 requires requests>=2.32.2, but you have requests 2.25.1 which is incompatible.
jupyterlab-server 2.27.3 requires requests>=2.31, but you have requests 2.25.1 which is incompatible.
tensorflow-probability 0.21.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.13.2 which is incompatible.


In [29]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Resize, Grayscale, ToTensor, Normalize
from torchattacks import FGSM, PGD

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data
transform = Compose([Resize(224), Grayscale(3), ToTensor(),
                     Normalize([0.5]*3, [0.5]*3)])
test_ds = MNIST("data", train=False, download=True, transform=transform)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False,
                         num_workers=4, pin_memory=True)

# 1. Load & wrap model
hf_model = ViTForImageClassification.from_pretrained(
    "farleyknight-org-username/vit-base-mnist"
).to(device).eval()

class ViTWrapper(nn.Module):
    def __init__(self, hf_model):
        super().__init__()
        self.hf_model = hf_model
    def forward(self, x):
        return self.hf_model(pixel_values=x).logits

wrapped_model = ViTWrapper(hf_model).to(device).eval()

# 2. Attacks
fgsm_attack = FGSM(wrapped_model, eps=0.1)
pgd_attack  = PGD(wrapped_model, eps=8/255, alpha=2/255, steps=5,
                  random_start=True)

# 3. Evaluation helper
def evaluate_attack(attack, name):
    correct, total = 0, 0
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        x_adv = attack(x, y)
        with torch.no_grad():
            logits = hf_model(pixel_values=x_adv).logits
        preds = logits.argmax(dim=-1)
        correct += (preds == y).sum().item()
        total   += y.size(0)
    print(f"{name} accuracy: {correct/total:.2%}")

# 4. Run evaluations
print("Clean accuracy:")
evaluate_attack(lambda x,y: x, "Clean")  # identity attack prints baseline
evaluate_attack(fgsm_attack, "FGSM")
evaluate_attack(pgd_attack,  "PGD")


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [16]:
# In[5]:
def make_adv_loader(model, attack, loader, device):
    model.eval()
    X_adv, Y_adv = [], []
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        x_adv = attack(x, y)
        X_adv.append(x_adv.detach().cpu())
        Y_adv.append(y.detach().cpu())
    X_adv = torch.cat(X_adv)
    Y_adv = torch.cat(Y_adv)
    ds = TensorDataset(X_adv, Y_adv)
    return DataLoader(ds, batch_size=64, shuffle=False,
                      num_workers=4, pin_memory=(device.type=="cuda"))

# In[6]:
fgsm_loader = make_adv_loader(wrapped_model, fgsm_attack, test_loader, device)
pgd_loader  = make_adv_loader(wrapped_model, pgd_attack,  test_loader, device)


In [17]:
# In[7]:
class FeatureSqueezeDetector(nn.Module):
    def __init__(self, model): super().__init__(); self.model = model
    def score(self, x):
        x_s = torch.floor(x*15)/15
        p = F.softmax(self.model(x),   dim=-1)
        q = F.softmax(self.model(x_s), dim=-1)
        return (p-q).abs().sum(dim=1)

class AutoencoderDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3,16,3,2,1), nn.ReLU(),
            nn.Conv2d(16,8,3,2,1), nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8,16,3,2,1,1), nn.ReLU(),
            nn.ConvTranspose2d(16,3,3,2,1,1), nn.Tanh()
        )
    def forward(self, x):
        return self.decoder(self.encoder(x))
    def score(self, x):
        recon = self(x)
        return ((x-recon)**2).mean(dim=[1,2,3])

# move AE to device
ae_detector = AutoencoderDetector().to(device).eval()


In [20]:
# In[8]:
# Take first batch from FGSM loader
x_batch, _ = next(iter(fgsm_loader))
x_batch = x_batch.to(device, non_blocking=True)

fsd = FeatureSqueezeDetector(wrapped_model).to(device).eval()
fs_scores = fsd.score(x_batch)
ae_scores = ae_detector.score(x_batch)

print("Feature-Squeeze scores:", fs_scores[:5].detach().cpu().numpy())
print("Autoencoder scores:    ", ae_scores[:5].detach().cpu().numpy())


OutOfMemoryError: CUDA out of memory. Tried to allocate 38.00 MiB. GPU 0 has a total capacity of 15.99 GiB of which 0 bytes is free. Of the allocated memory 28.75 GiB is allocated by PyTorch, and 588.75 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [23]:
# In[8]: Evaluate detectors on clean, FGSM, and PGD sets

import numpy as np
import torch
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt

# assume these exist from prior cells:
#   test_loader      – clean MNIST DataLoader
#   fgsm_loader      – FGSM adversarial DataLoader
#   pgd_loader       – PGD adversarial DataLoader
#   fs_detector      – FeatureSqueezeDetector(wrapped_model).to(device).eval()
#   ae_detector      – AutoencoderDetector().to(device).eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collect_scores(detector, loader, sub_batch=16):
    detector.eval()
    all_scores = []
    with torch.no_grad():
        for x, _ in loader:
            x = x.to(device, non_blocking=True)
            scores = []
            for i in range(0, x.size(0), sub_batch):
                chunk = x[i:i+sub_batch]
                s = detector.score(chunk)
                scores.append(s.detach().cpu())
            all_scores.append(torch.cat(scores))
            torch.cuda.empty_cache()
    return torch.cat(all_scores).numpy()

# In[9]: Instantiate the Feature‐Squeeze detector
fs_detector = FeatureSqueezeDetector(wrapped_model).to(device).eval()


# 1) Collect scores
fs_clean = collect_scores(fs_detector, test_loader)
fs_fgsm  = collect_scores(fs_detector, fgsm_loader)
fs_pgd   = collect_scores(fs_detector, pgd_loader)

ae_clean = collect_scores(ae_detector, test_loader)
ae_fgsm  = collect_scores(ae_detector, fgsm_loader)
ae_pgd   = collect_scores(ae_detector, pgd_loader)

# 2) Build labels & score arrays
y_clean = np.zeros_like(fs_clean)
y_adv   = np.ones_like (fs_fgsm)      # we’ll treat FGSM/PGD both as positive
y_true  = np.concatenate([y_clean, y_adv, y_adv])

fs_scores = np.concatenate([fs_clean, fs_fgsm, fs_pgd])
ae_scores = np.concatenate([ae_clean, ae_fgsm, ae_pgd])

# 3) ROC & AUC
fpr_fs, tpr_fs, _ = roc_curve(y_true, fs_scores, pos_label=1)
roc_auc_fs       = auc(fpr_fs, tpr_fs)
fpr_ae, tpr_ae, _ = roc_curve(y_true, ae_scores, pos_label=1)
roc_auc_ae       = auc(fpr_ae, tpr_ae)

plt.figure(figsize=(6,4))
plt.plot(fpr_fs, tpr_fs, label=f'Feature-Squeeze (AUC={roc_auc_fs:.2f})')
plt.plot(fpr_ae, tpr_ae, label=f'Autoencoder  (AUC={roc_auc_ae:.2f})')
plt.plot([0,1],[0,1],'k--',alpha=0.3)
plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
plt.title('ROC Curves'); plt.legend(loc='lower right')
plt.show()

# 4) Precision–Recall & AP
prec_fs, rec_fs, _ = precision_recall_curve(y_true, fs_scores, pos_label=1)
ap_fs             = auc(rec_fs, prec_fs)
prec_ae, rec_ae, _ = precision_recall_curve(y_true, ae_scores, pos_label=1)
ap_ae             = auc(rec_ae, prec_ae)

plt.figure(figsize=(6,4))
plt.plot(rec_fs, prec_fs, label=f'FS (AP={ap_fs:.2f})')
plt.plot(rec_ae, prec_ae, label=f'AE (AP={ap_ae:.2f})')
plt.xlabel('Recall'); plt.ylabel('Precision')
plt.title('Precision–Recall Curves'); plt.legend(loc='lower left')
plt.show()


RuntimeError: Caught RuntimeError in pin memory thread for device 0.
Original Traceback (most recent call last):
  File "c:\Users\Abhis\.conda\envs\torch_gpu_env\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 38, in do_one_step
    data = pin_memory(data, device)
  File "c:\Users\Abhis\.conda\envs\torch_gpu_env\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 89, in pin_memory
    clone[i] = pin_memory(item, device)
  File "c:\Users\Abhis\.conda\envs\torch_gpu_env\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 59, in pin_memory
    return data.pin_memory(device)
RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

